<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Carga

En este laboratorio, tu objetivo será diseñar la estructura de una base de datos relacional, crear las tablas necesarias y cargar en ellas los datos limpios y preparados que obtuviste en el laboratorio anterior. Trabajarás con datos relacionados con la demanda, generación eléctrica, y variables socioeconómicas, almacenándolos de manera eficiente para facilitar el análisis y las consultas posteriores.





In [1]:
# Manipulación de DataFrames y variables de entorno
# -----------------------------------------------------------------------
import pandas as pd                             # Manipulación y análisis de datos en estructuras DataFrame
import dotenv                                   # Manejo de variables de entorno (.env)
import os                                       # Interacción con el sistema operativo (lectura de variables de entorno)
from datetime import datetime                   # Importación específica de datetime para facilitar manejo de fechas
pd.set_option('display.max_columns', None)
import sys
sys.path.append("../")                          # Añadir rutas externas para importar módulos desde carpetas superiores
from src.SupportCreacionDB import (
    conectarse_a_bd,
    modificar_bd,
    insertar_muchos_datos_bd,
    generar_tupla,
)

In [17]:
df_demanda = pd.read_csv("datos\DatosFinales\RedElectrica\DemandaElectrica.csv",index_col="Unnamed: 0")
df_generacion = pd.read_csv("datos\DatosFinales\RedElectrica\GeneracionElectrica.csv",index_col="Unnamed: 0")
df_demograficos = pd.read_csv("datos\DatosFinales\INE\DatosDemograficos2019-2021.csv",index_col="Unnamed: 0")
df_pib = pd.read_csv("datos\DatosFinales\INE\DatosPIB2019-2021.csv",index_col="Unnamed: 0")

- **Diseño de la Base de Datos:** Define una estructura de base de datos relacional que sea adecuada para almacenar los datos de demanda eléctrica, generación eléctrica, población y PIB por provincia.


- **Creación de la Base de Datos:** Escribe los scripts SQL necesarios para crear las tablas definidas en la estructura de la base de datos. Asegúrate de definir correctamente los tipos de datos y las restricciones (e.g., `NOT NULL`, `UNIQUE`).


- **Carga de Datos en la Base de Datos:** Utiliza Python para cargar los datos limpios y preparados en las tablas de la base de datos. Asegúrate de que los datos se insertan correctamente, manejando posibles errores o inconsistencias en los datos.

# Unicas
- comunidad
- provincia
- año
- mes

### año

In [7]:
key = os.getenv("sql")
user = "postgres"
db = "modulo5"

In [19]:
conexion = conectarse_a_bd(db,user,key)
cursor = conexion.cursor()

Conectado a la base de datos: modulo5


In [22]:
df_demanda = pd.read_csv("datos\DatosFinales\RedElectrica\DemandaElectrica.csv",index_col="Unnamed: 0")
df_years = pd.DataFrame({"anio" : df_demanda["año"].unique()})
df_months = pd.DataFrame({"mes" : df_demanda["mes"].unique()})

In [23]:
df_years.index = df_years.index + 1
df_years.reset_index(inplace=True)
df_years

,index,anio
0,1,2019
1,2,2020
2,3,2021


In [24]:
df_months.index = df_months.index + 1
df_months.reset_index(inplace=True)
df_months

,index,mes
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
5,6,6
6,7,7
7,8,8
8,9,9
9,10,10


In [31]:
tupla_years = generar_tupla(df_years)
tupla_month = generar_tupla(df_months)

# Creamos años

In [29]:
conexion = conectarse_a_bd(db,user,key)
query = """CREATE TABLE IF NOT EXISTS years (
            id_year INT PRIMARY KEY,
            year INT UNIQUE NOT NULL
            );"""
modificar_bd(conexion,query)

Conectado a la base de datos: modulo5
Se ha modificado correctamente la base de Datos


### Insertamos datos

In [11]:
conexion = conectarse_a_bd(db,user,key)
query = """ INSERT INTO years(id_year,year) VALUES (%s,%s)"""
insertar_muchos_datos_bd(conexion,query,tupla_years)

Conectado a la base de datos: modulo5
Se han añadido los valores correctamente


# Creamos años

In [32]:
conexion = conectarse_a_bd(db,user,key)
query = """CREATE TABLE IF NOT EXISTS month (
            id_month INT PRIMARY KEY,
            month INT UNIQUE NOT NULL
            );"""
modificar_bd(conexion,query)

Conectado a la base de datos: modulo5
Se ha modificado correctamente la base de Datos


### Insertamos datos

In [33]:
conexion = conectarse_a_bd(db,user,key)
query = """ INSERT INTO month(id_month,month) VALUES (%s,%s)"""
insertar_muchos_datos_bd(conexion,query,tupla_month)

Conectado a la base de datos: modulo5
Se han añadido los valores correctamente


# Creamos comunidades_autonomas

In [2]:
df_demograficos = pd.read_csv("datos\DatosFinales\INE\DatosDemograficos2019-2021.csv",index_col="Unnamed: 0")

In [3]:
df_demograficos.sample()

,comunidad_autonoma,provincias,edad (3 grupos de edad),españoles/extranjeros,sexo,año,total
1032,Galicia,Lugo,65 y más,Españoles,Hombres,2021,42259


In [4]:
df_ccaas = pd.DataFrame({"ccaa" : df_demograficos["comunidad_autonoma"].unique()})

In [5]:
df_ccaas.index = df_ccaas.index + 1
df_ccaas.reset_index(inplace=True)
df_ccaas

,index,ccaa
0,1,Castilla-La Mancha
1,2,Comunidad Valenciana
2,3,Andalucía
3,4,País Vasco
4,5,Asturias
5,6,Castilla y León
6,7,Extremadura
7,8,Islas Baleares
8,9,Cataluña
9,10,Cantabria


In [8]:
conexion = conectarse_a_bd(db,user,key)
query = """CREATE TABLE IF NOT EXISTS comunidades_autonomas (
            id_ccaas INT PRIMARY KEY,
            comunidad_autonoma VARCHAR(100) UNIQUE NOT NULL
            );"""
modificar_bd(conexion,query)

Conectado a la base de datos: modulo5
Se ha modificado correctamente la base de Datos


In [9]:
tupla_ccaas = generar_tupla(df_ccaas)

### Insertamos Datos

In [12]:
conexion = conectarse_a_bd(db,user,key)
query = """ INSERT INTO comunidades_autonomas(id_ccaas,comunidad_autonoma) VALUES (%s,%s)"""
insertar_muchos_datos_bd(conexion,query,tupla_ccaas)

Conectado a la base de datos: modulo5
Se han añadido los valores correctamente


# Creamos provincias

In [21]:
df_provincias = pd.DataFrame({"provincias":df_pib["provincias"].unique()})

In [22]:
df_provincias.index = df_provincias.index + 1
df_provincias.reset_index(inplace=True)


In [ ]:
conexion = conectarse_a_bd(db,user,key)
query = """CREATE TABLE IF NOT EXISTS provincias (
            id_provincia INT PRIMARY KEY,
            provincia VARCHAR(100) UNIQUE NOT NULL
            );"""
modificar_bd(conexion,query)

### Insertamos Datos